## Birdspotter is a keras machine learning model to identify images that contain birds

Written by George Bigham


In [11]:
import keras
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
from keras.preprocessing.image import ImageDataGenerator

Code for moving image files

In [12]:
import shutil
import os

source = 'images/'

# names is a list of subdirectories (categories) in the images folder
names = os.listdir(source)
# names

Data split 80% for training and 20% validation

In [19]:
# move 80% of files to training set and rest to validation set

train_set = 'data/train/bird/'
valid_set = 'data/validation/bird/'

for name in names:
    if name != '.DS_Store':
        files = os.listdir(source + name)
        n_files = len(files)
        train_set_size = 0.8 * n_files
        train_file_count = 0
        for file in files:
            if train_file_count < train_set_size:
                shutil.copy(source + name + '/' + file, train_set)
            else:
                shutil.copy(source + name + '/' + file, valid_set)
            train_file_count += 1    

In [13]:
# rename images in 101_ObjectCategories to be unique across categories
non_bird_source = '101_ObjectCategories'
object_categories = os.listdir(non_bird_source)

for object_type in object_categories:
    if object_type != '.DS_Store':
        files = os.listdir(non_bird_source + '/'+ object_type)
        for file in files:
            os.rename(non_bird_source + '/'+ object_type + '/' + file, \
                      non_bird_source + '/'+ object_type + '/' + object_type +'_' + file)

In [21]:
# move non bird images to birdless folder in training and validation

non_bird_source = '101_ObjectCategories'
object_categories = os.listdir(non_bird_source)

train_set_nb = 'data/train/birdless/'
valid_set_nb = 'data/validation/birdless/'


for object_type in object_categories:
    if object_type != '.DS_Store':
        files = os.listdir(non_bird_source + '/'+ object_type)
        n_files = len(files)
        file_count = 0
        for file in files:
            if file_count < 40:
                shutil.copy(non_bird_source + '/'+ object_type + '/' + file, train_set_nb)
            elif ((file_count >= 40) & (file_count <60)):
                shutil.copy(non_bird_source +'/'+ object_type + '/' + file, valid_set_nb)
            else: break
            file_count += 1 

## Code for building machine learning model


In [13]:
from keras.preprocessing.image import ImageDataGenerator

# initalize Image Data Generator for image augmentation

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [14]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [15]:
# build model

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [16]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [17]:
batch_size = 16

# this is the augmentation configuration used for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'data/train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'data/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

Found 8911 images belonging to 2 classes.
Found 2474 images belonging to 2 classes.


In [27]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)
model.save('first_try_full.h5')

Epoch 1/50
125/125 [==============================] - 26s 205ms/step - loss: 0.6966 - accuracy: 0.6130 - val_loss: 0.5452 - val_accuracy: 0.6925
Epoch 2/50
125/125 [==============================] - 24s 191ms/step - loss: 0.6149 - accuracy: 0.6813 - val_loss: 0.5862 - val_accuracy: 0.7525
Epoch 3/50
125/125 [==============================] - 24s 194ms/step - loss: 0.5655 - accuracy: 0.7320 - val_loss: 0.5584 - val_accuracy: 0.6587
Epoch 4/50
125/125 [==============================] - 25s 199ms/step - loss: 0.5738 - accuracy: 0.7250 - val_loss: 0.4723 - val_accuracy: 0.7733
Epoch 5/50
125/125 [==============================] - 26s 208ms/step - loss: 0.5290 - accuracy: 0.7535 - val_loss: 0.6079 - val_accuracy: 0.6825
Epoch 6/50
125/125 [==============================] - 26s 206ms/step - loss: 0.5298 - accuracy: 0.7494 - val_loss: 0.3508 - val_accuracy: 0.7750
Epoch 7/50
125/125 [==============================] - 26s 204ms/step - loss: 0.5128 - accuracy: 0.7850 - val_loss: 0.4316 - val_ac

Epoch 1/200
62/62 [==============================] - 20s 315ms/step - loss: 0.6661 - accuracy: 0.8800 - val_loss: 0.2256 - val_accuracy: 0.9500
Epoch 2/200
62/62 [==============================] - 19s 309ms/step - loss: 0.5218 - accuracy: 0.8850 - val_loss: 0.1277 - val_accuracy: 0.9550
Epoch 3/200
62/62 [==============================] - 19s 309ms/step - loss: 0.3247 - accuracy: 0.8901 - val_loss: 0.3132 - val_accuracy: 0.8900
Epoch 4/200
62/62 [==============================] - 19s 307ms/step - loss: 0.3445 - accuracy: 0.8881 - val_loss: 1.8885 - val_accuracy: 0.7000
Epoch 5/200
62/62 [==============================] - 19s 309ms/step - loss: 0.5100 - accuracy: 0.8810 - val_loss: 0.2382 - val_accuracy: 0.9475
Epoch 6/200
62/62 [==============================] - 19s 302ms/step - loss: 0.3101 - accuracy: 0.8962 - val_loss: 0.2304 - val_accuracy: 0.9350
Epoch 7/200
62/62 [==============================] - 19s 307ms/step - loss: 0.4098 - accuracy: 0.8962 - val_loss: 0.1387 - val_accuracy:

62/62 [==============================] - 18s 298ms/step - loss: 0.3677 - accuracy: 0.8780 - val_loss: 0.1558 - val_accuracy: 0.9575
Epoch 114/200
62/62 [==============================] - 18s 296ms/step - loss: 0.3895 - accuracy: 0.8810 - val_loss: 0.1439 - val_accuracy: 0.9475
Epoch 115/200
62/62 [==============================] - 19s 301ms/step - loss: 0.4335 - accuracy: 0.8720 - val_loss: 0.2119 - val_accuracy: 0.9525
Epoch 116/200
62/62 [==============================] - 19s 302ms/step - loss: 0.4869 - accuracy: 0.8639 - val_loss: 0.7276 - val_accuracy: 0.8625
Epoch 117/200
62/62 [==============================] - 20s 315ms/step - loss: 0.4246 - accuracy: 0.8911 - val_loss: 0.2146 - val_accuracy: 0.9475
Epoch 118/200
62/62 [==============================] - 19s 312ms/step - loss: 0.3956 - accuracy: 0.8931 - val_loss: 0.5284 - val_accuracy: 0.9275
Epoch 119/200
62/62 [==============================] - 19s 312ms/step - loss: 0.6084 - accuracy: 0.8881 - val_loss: 0.1993 - val_accuracy:

In [30]:
# hard coded batch size of 32
model.fit_generator(
        train_generator,
        steps_per_epoch=1000 // 32,
        epochs=200,
        validation_data=validation_generator,
        validation_steps=400 // 32)
model.save('third_try_full.h5')


Epoch 1/200
31/31 [==============================] - 7s 218ms/step - loss: 0.5631 - accuracy: 0.8165 - val_loss: 0.2328 - val_accuracy: 0.8229
Epoch 2/200
31/31 [==============================] - 6s 199ms/step - loss: 0.3732 - accuracy: 0.8488 - val_loss: 0.3416 - val_accuracy: 0.7969
Epoch 3/200
31/31 [==============================] - 6s 191ms/step - loss: 0.4019 - accuracy: 0.8548 - val_loss: 0.3922 - val_accuracy: 0.7396
Epoch 4/200
31/31 [==============================] - 6s 186ms/step - loss: 0.4263 - accuracy: 0.8347 - val_loss: 1.2577 - val_accuracy: 0.8594
Epoch 5/200
31/31 [==============================] - 6s 181ms/step - loss: 0.7764 - accuracy: 0.8165 - val_loss: 0.6492 - val_accuracy: 0.8438
Epoch 6/200
31/31 [==============================] - 6s 183ms/step - loss: 0.5355 - accuracy: 0.8327 - val_loss: 0.4214 - val_accuracy: 0.8229
Epoch 7/200
31/31 [==============================] - 6s 190ms/step - loss: 0.4546 - accuracy: 0.8468 - val_loss: 0.3221 - val_accuracy: 0.8646

Epoch 58/200
31/31 [==============================] - 6s 195ms/step - loss: 0.4124 - accuracy: 0.8327 - val_loss: 0.2109 - val_accuracy: 0.8490
Epoch 59/200
31/31 [==============================] - 6s 189ms/step - loss: 0.4098 - accuracy: 0.8508 - val_loss: 0.3146 - val_accuracy: 0.8802
Epoch 60/200
31/31 [==============================] - 6s 187ms/step - loss: 0.4723 - accuracy: 0.8105 - val_loss: 1.2411 - val_accuracy: 0.7188
Epoch 61/200
31/31 [==============================] - 5s 177ms/step - loss: 0.4474 - accuracy: 0.8609 - val_loss: 19.3193 - val_accuracy: 0.7760
Epoch 62/200
31/31 [==============================] - 5s 175ms/step - loss: 0.4579 - accuracy: 0.8427 - val_loss: 0.4118 - val_accuracy: 0.7656
Epoch 63/200
31/31 [==============================] - 6s 179ms/step - loss: 0.4076 - accuracy: 0.8528 - val_loss: 0.2650 - val_accuracy: 0.8021
Epoch 64/200
31/31 [==============================] - 5s 175ms/step - loss: 0.5088 - accuracy: 0.8306 - val_loss: 0.3962 - val_accuracy

Epoch 115/200
31/31 [==============================] - 6s 190ms/step - loss: 0.4637 - accuracy: 0.8306 - val_loss: 0.6106 - val_accuracy: 0.8698
Epoch 116/200
31/31 [==============================] - 6s 182ms/step - loss: 0.5696 - accuracy: 0.8044 - val_loss: 0.2759 - val_accuracy: 0.8854
Epoch 117/200
31/31 [==============================] - 6s 180ms/step - loss: 0.4220 - accuracy: 0.8488 - val_loss: 0.4553 - val_accuracy: 0.8817
Epoch 118/200
31/31 [==============================] - 6s 178ms/step - loss: 0.4075 - accuracy: 0.8384 - val_loss: 0.1629 - val_accuracy: 0.8958
Epoch 119/200
31/31 [==============================] - 5s 177ms/step - loss: 0.4837 - accuracy: 0.8246 - val_loss: 0.4454 - val_accuracy: 0.6771
Epoch 120/200
31/31 [==============================] - 6s 180ms/step - loss: 0.5417 - accuracy: 0.8226 - val_loss: 0.2032 - val_accuracy: 0.8542
Epoch 121/200
31/31 [==============================] - 5s 177ms/step - loss: 0.3947 - accuracy: 0.8427 - val_loss: 0.8029 - val_ac

In [20]:
# hard coded batch size of 32
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // 32,
        epochs=400,
        validation_data=validation_generator,
        validation_steps=400 // 32)
model.save('fourth_try_full.h5')

Epoch 1/400
62/62 [==============================] - 12s 197ms/step - loss: 0.4598 - accuracy: 0.8448 - val_loss: 0.3532 - val_accuracy: 0.8802
Epoch 2/400
62/62 [==============================] - 11s 182ms/step - loss: 0.3946 - accuracy: 0.8347 - val_loss: 0.4305 - val_accuracy: 0.8333
Epoch 3/400
62/62 [==============================] - 11s 184ms/step - loss: 0.4107 - accuracy: 0.8427 - val_loss: 0.2582 - val_accuracy: 0.8229
Epoch 4/400
62/62 [==============================] - 11s 183ms/step - loss: 0.3855 - accuracy: 0.8427 - val_loss: 0.5369 - val_accuracy: 0.8854
Epoch 5/400
62/62 [==============================] - 13s 206ms/step - loss: 0.3747 - accuracy: 0.8619 - val_loss: 0.3428 - val_accuracy: 0.9062
Epoch 6/400
62/62 [==============================] - 12s 199ms/step - loss: 0.3378 - accuracy: 0.8599 - val_loss: 0.1136 - val_accuracy: 0.8698
Epoch 7/400
62/62 [==============================] - 12s 200ms/step - loss: 0.4352 - accuracy: 0.8528 - val_loss: 0.2015 - val_accuracy:

62/62 [==============================] - 11s 184ms/step - loss: 0.3124 - accuracy: 0.8740 - val_loss: 0.3003 - val_accuracy: 0.8906
Epoch 114/400
62/62 [==============================] - 11s 182ms/step - loss: 0.3035 - accuracy: 0.8931 - val_loss: 0.5046 - val_accuracy: 0.8802
Epoch 115/400
62/62 [==============================] - 12s 189ms/step - loss: 0.3061 - accuracy: 0.8780 - val_loss: 0.0107 - val_accuracy: 0.9062
Epoch 116/400
62/62 [==============================] - 12s 193ms/step - loss: 0.2938 - accuracy: 0.8881 - val_loss: 0.2594 - val_accuracy: 0.7865
Epoch 117/400
62/62 [==============================] - 11s 185ms/step - loss: 0.4190 - accuracy: 0.8841 - val_loss: 0.2285 - val_accuracy: 0.8710
Epoch 118/400
62/62 [==============================] - 11s 179ms/step - loss: 0.3115 - accuracy: 0.8851 - val_loss: 0.0966 - val_accuracy: 0.8906
Epoch 119/400
62/62 [==============================] - 12s 191ms/step - loss: 0.2932 - accuracy: 0.8851 - val_loss: 0.1259 - val_accuracy:

62/62 [==============================] - 12s 195ms/step - loss: 0.3474 - accuracy: 0.8942 - val_loss: 0.1937 - val_accuracy: 0.9167
Epoch 226/400
62/62 [==============================] - 12s 197ms/step - loss: 0.3695 - accuracy: 0.9032 - val_loss: 0.1364 - val_accuracy: 0.8750
Epoch 227/400
62/62 [==============================] - 12s 187ms/step - loss: 0.3531 - accuracy: 0.9032 - val_loss: 0.2122 - val_accuracy: 0.9479
Epoch 228/400
62/62 [==============================] - 12s 187ms/step - loss: 0.3230 - accuracy: 0.8952 - val_loss: 0.0137 - val_accuracy: 0.9427
Epoch 229/400
62/62 [==============================] - 12s 192ms/step - loss: 0.3487 - accuracy: 0.8799 - val_loss: 0.1403 - val_accuracy: 0.9323
Epoch 230/400
62/62 [==============================] - 12s 193ms/step - loss: 0.3271 - accuracy: 0.8891 - val_loss: 0.2076 - val_accuracy: 0.8802
Epoch 231/400
62/62 [==============================] - 12s 196ms/step - loss: 0.3120 - accuracy: 0.8911 - val_loss: 0.3842 - val_accuracy:

62/62 [==============================] - 12s 190ms/step - loss: 0.4356 - accuracy: 0.8579 - val_loss: 0.3360 - val_accuracy: 0.9375
Epoch 338/400
62/62 [==============================] - 11s 181ms/step - loss: 0.4537 - accuracy: 0.8629 - val_loss: 0.1187 - val_accuracy: 0.9688
Epoch 339/400
62/62 [==============================] - 12s 190ms/step - loss: 0.4772 - accuracy: 0.8528 - val_loss: 3.2029 - val_accuracy: 0.8594
Epoch 340/400
62/62 [==============================] - 12s 187ms/step - loss: 0.4502 - accuracy: 0.8679 - val_loss: 0.3428 - val_accuracy: 0.9323
Epoch 341/400
62/62 [==============================] - 12s 196ms/step - loss: 0.3060 - accuracy: 0.8911 - val_loss: 0.0791 - val_accuracy: 0.9167
Epoch 342/400
62/62 [==============================] - 12s 199ms/step - loss: 0.3257 - accuracy: 0.8819 - val_loss: 0.0071 - val_accuracy: 0.9688
Epoch 343/400
62/62 [==============================] - 13s 202ms/step - loss: 0.4376 - accuracy: 0.9133 - val_loss: 0.0755 - val_accuracy:

In [21]:
# Load the model

from tensorflow.keras.models import load_model
model = load_model("fourth_try_full.h5")
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [22]:
import numpy as np
from keras.preprocessing import image

img_width, img_height = 150, 150


In [25]:
img = image.load_img('test/bird1.jpg', target_size = (150, 150))
img = image.img_to_array(img)
img = img*(1./256)  # rescale
img = np.expand_dims(img, axis = 0)

confident_bird = 1- model.predict(img, batch_size = 1)
confident_bird

array([[1.]], dtype=float32)

In [28]:
confident_bird[0][0]

1.0

In [ ]:
# legacy code for model hpyer-parameters 
model.fit_generator(
        train_generator,
        steps_per_epoch=1000 // batch_size,
        epochs=200,
        validation_data=validation_generator,
        validation_steps=400 // batch_size)
model.save('second_try_full.h5')